In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = '/kaggle/input/mlda-hw4/Train.csv'
train_data = pd.read_csv(path, parse_dates=['date'])

train_data.head()

In [ ]:
train_data['date'] = (train_data['date'].dt.year - 2011) * 12 + train_data['date'].dt.month

In [ ]:
train_data['date']

**Предобрабокта данных**

In [ ]:
train_data['date'] = train_data['date'].astype(int).div(10**9)

In [ ]:
train_data['date']

In [ ]:
train_data = train_data.drop(columns=['id'])


In [ ]:
train_data.isnull().sum()

Всего в данных 100к записей => в колонках build_tech и g_lift отсутствует почти треть данных.

In [ ]:
train_data

Попробуем заполнить NaN медианной по каждому столбцу

In [ ]:
cols = ['kw1', 'kw2', 'kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13']

for col in cols:
    print(train_data[col].value_counts())
    print('--------------------------------')

In [ ]:
import seaborn as sns

# sns.pairplot(train_data)

Можно заметить, что при значении build_tech=1.0 самые дорогие квартиры, а также при встрече kw3...kw12 стоимость квартирты ниже, чем при их отсутствии. Возможно можно будет схлопнуть их в один столбец
Обзорная диаграмма, чтобы выделить зависимости, дальше построим подробнее

In [ ]:
# cols = ['kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13']
# train_data['sum_kw_3_13'] = train_data[cols].sum(axis=1)
# train_data = train_data.drop(columns=cols)

In [ ]:
train_data

**Первый подход, просто обучим catboostRegressor и посмотрим на shap_values, которые получатся**

In [ ]:
X_test.dtypes

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['price'])
y = train_data['price']

cat_features = ['street_id', 'balcon', 'rooms', 'date']

#Замени пропущенные значения на медиану по столбцу

X = X.fillna(X.median())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

model = CatBoostRegressor(iterations=6000,
                         depth=6,
                         custom_metric=['MAE'],
                         cat_features=cat_features)

eval_dataset = Pool(X_test,
                   y_test,
                   cat_features=['street_id', 'balcon', 'rooms'])

model.fit(X_train,
         y_train,
         eval_set=eval_dataset,
         verbose=500)

In [ ]:
test_df = pd.read_csv('/kaggle/input/mlda-hw4/Train.csv', parse_dates=['date'])

X_test_set = test_df.drop(columns=['id'])

# cols = ['kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13']
# X_test_set['sum_kw_3_13'] = X_test_set[cols].sum(axis=1)
# X_test_set = X_test_set.drop(columns=cols)
X_test_set['date'] = X_test_set['date'].astype(int).div(10**9)

preds = model.predict(X_test_set)

In [ ]:
idx = []
pred = []
for i in range(len(preds)):
    idx.append(100000 + i)
    pred.append(preds[i])

result = pd.DataFrame({'id': idx, 'price': pred})
result.to_csv('with_cat_feat_popitka.csv', index=False)

In [ ]:
result